In [ ]:
import pandas as pd
import statsapi
import datetime
import string
import json
import os

from bs4 import BeautifulSoup
import requests

In [ ]:
# Boolean for advantageous matchups (L vs R)
def adv_matchup(row):
    val = 0
    if row['batter_hand'] == 'S':
        val = 1
    elif row['batter_hand'] == 'R' and row['pitcher_hand'] == 'L':
        val = 1
    elif row['batter_hand'] == 'L' and row['pitcher_hand'] == 'R':
        val = 1
    return val

for file in os.scandir("data\\matchups"):
    matchups = pd.read_csv(file, index_col=0)
    matchups['adv_ab'] = matchups.apply(adv_matchup, axis=1)
    matchups.to_csv(f"data\\matchups\\{file.name}")


In [ ]:
# Change switch hitters to side
def fix_switch(row):
    hand = row['batter_hand']
    if hand == 'S' and row['pitcher_hand'] == 'R':
        hand = 'L'
    elif hand == 'S' and row['pitcher_hand'] == 'L':
        hand = 'R'
    return(hand)

matchups['batter_hand'] = matchups.apply(fix_switch, axis=1)
matchups.to_csv("data\\matchups\\all_matchups.csv")
